**Implementing PyPDF 2 with BMO Report - Q2 Results**

In [ ]:
!pip install PyPDF2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.7 MB/s eta 0:00:00


In [ ]:
import PyPDF2

pdfFileObj = open('/content/BMO Report.pdf','rb')
pdfReader = PyPDF2.PdfReader(pdfFileObj)
print(pdfReader.pages)
pageObj = pdfReader.pages[0]
print(pageObj.extract_text())
pdfFileObj.close()

BMO Financial Group Reports Second Quarter 2024 Results
EARNINGS RELEASE
BMO’s Second Quarter 2024 Report to Shareholders, including the unaudited interim consolidated financial statements for the period ended 
April 30, 2024 are available online at www.bmo.com/investorrelations and at www.sedarplus.ca . 
Financial Results Highlights 
Second Quarter 2024 compared with Second Quarter 2023:
•Net income of $1,866 million, com pared w ith $1,029 million; adjusted n et income1, 2 of $2,033  million, compared with  $2,186 million
•Repor ted earning s per share (EPS)3 of $2.36,  compare d with $1.2 6; adjusted EPS1, 2, 3 of $2.59,  compa red with  $2.89
•Provis ion for credit losses (PCL) of $70 5 million, c ompared w ith $1,023 m illion on a reporte d basis and $318 million on an adjusted basis1
•Return on equity (ROE) of 9.9%,  compare d with 5.5%; adjusted ROE1, 2 of 10.9%, compa red with 12.6 %
•Common Equity Tier 1 (CET1) Ratio4 of 13.1%, compared w ith 12.2%
•Declared a quarterl y divid

**Import PDFMiner - An Alternative**

In [ ]:
!pip install pdfminer.six

In [ ]:
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser
import io

**IS PDF Resource Manager Obsolete?**

In [ ]:
def extract_text_from_pdf(pdf_path):
    resource_manager = PDFResourceManager()
    fake_file_handle = io.StringIO()
    converter = TextConverter(resource_manager, fake_file_handle)
    page_interpreter = PDFPageInterpreter(resource_manager, converter)
    with open(pdf_path, 'rb') as fh:
        for page in PDFPage.get_pages(fh,
                                      caching=True,
                                      check_extractable=True):
            page_interpreter.process_page(page)
        text = fake_file_handle.getvalue()
    # close open handles
    converter.close()
    fake_file_handle.close()
    if text:
      return text

In [ ]:
extract_text_from_pdf('/content/BMO Report.pdf')

"BMO Financial Group Reports Second Quarter 2024 ResultsEARNINGS RELEASEBMO’s Second Quarter\xa02024 Report to Shareholders, including the unaudited interim consolidated financial statements for the period ended April\xa030,\xa02024 are available online at www.bmo.com/investorrelations and at www.sedarplus.ca. Financial Results Highlights Second Quarter 2024 compared with Second Quarter 2023:•Net income of $1,866\xa0million, compared with\xa0$1,029\xa0million; adjusted net income1, 2 of $2,033\xa0million, compared with $2,186\xa0million•Reported earnings per share (EPS)3 of\xa0$2.36, compared with\xa0$1.26; adjusted EPS1, 2, 3 of\xa0$2.59, compared with\xa0$2.89•Provision for credit losses (PCL) of $705\xa0million, compared with $1,023\xa0million on a reported basis and $318 million on an adjusted basis1•Return on equity (ROE) of\xa09.9%, compared with\xa05.5%; adjusted ROE1, 2 of 10.9%, compared with\xa012.6%•Common Equity Tier\xa01 (CET1) Ratio4 of\xa013.1%, compared with\xa012.2%•De

**Lets Explore PDFMiner.six**

In [ ]:
from pdfminer.high_level import extract_text
def extract_text_from_pdf(pdf_path):
    laparams = LAParams()
    text = extract_text(pdf_path,laparams)
    return text.split("\f")

In [ ]:
extract_text_from_pdf('/content/BMO Report.pdf')

["BMO Financial Group Reports Second Quarter 2024 Results\n\nEARNINGS RELEASE\nBMO’s Second Quarter\xa02024 Report to Shareholders, including the unaudited interim consolidated financial statements for the period ended \nApril\xa030,\xa02024 are available online at www.bmo.com/investorrelations and at www.sedarplus.ca. \n\nFinancial Results Highlights \nSecond Quarter 2024 compared with Second Quarter 2023:\n• Net income of $1,866\xa0million, compared with\xa0$1,029\xa0million; adjusted net income1, 2 of $2,033\xa0million, compared with $2,186\xa0million\n•\n•\n•\n•\n•\n\nReported earnings per share (EPS)3 of\xa0$2.36, compared with\xa0$1.26; adjusted EPS1, 2, 3 of\xa0$2.59, compared with\xa0$2.89\nProvision for credit losses (PCL) of $705\xa0million, compared with $1,023\xa0million on a reported basis and $318 million on an adjusted basis1\nReturn on equity (ROE) of\xa09.9%, compared with\xa05.5%; adjusted ROE1, 2 of 10.9%, compared with\xa012.6%\nCommon Equity Tier\xa01 (CET1) Ratio4

**But Can it extract text across pages?**

In [ ]:
import re
def extract_text_between_words_across_pages(text_pages, start_word, end_word):
    extracted_texts = []
    combined_text = "\n".join(text_pages)
    pattern = re.compile(re.escape(start_word) + '(.*?)' + re.escape(end_word), re.S)

    for match in pattern.finditer(combined_text):
        start_page, end_page = None, None
        start_index, end_index = match.start(), match.end()

        cumulative_length = 0
        for i, page in enumerate(text_pages):
            page_length = len(page)
            if start_page is None and start_index < cumulative_length + page_length:
                start_page = i + 1
            if end_page is None and end_index <= cumulative_length + page_length:
                end_page = i + 1
            cumulative_length += page_length + len("\n")
            if start_page is not None and end_page is not None:
                break

        extracted_texts.append({
            'start_page': start_page,
            'end_page': end_page,
            'start_word': start_word,
            'end_word': end_word,
            'extracted_text': match.group(1).strip()
        })
    return extracted_texts


In [ ]:
start_end_pairs = [
    ('Regulatory Filings', 'Bank'),
    # Add more pairs as needed
]

import os
def process_files_in_folder(file_name, start_end_pairs):
    all_extracted_texts = []
    if file_name.endswith('.pdf'):
      text_pages = extract_text_from_pdf(file_name)
      for start_word, end_word in start_end_pairs:
        extracted_texts = extract_text_between_words_across_pages(text_pages, start_word, end_word)
        all_extracted_texts.extend(extracted_texts)

    return all_extracted_texts

all_extracted_texts = process_files_in_folder('/content/BMO Report.pdf', start_end_pairs)

In [ ]:
all_extracted_texts

[{'start_page': 3,
  'end_page': 3,
  'start_word': 'Regulatory Filings',
  'end_word': 'Bank',
  'extracted_text': 'BMO’s continuous disclosure materials, including interim filings, annual Management’s Discussion and Analysis and audited annual consolidated \nfinancial statements, Annual Information Form and Notice of Annual Meeting of Shareholders and Proxy Circular, are available on our website at \nwww.bmo.com/investorrelations, on the Canadian Securities Administrators’ website at www.sedarplus.ca, and on the EDGAR section of the \nU.S.\xa0Securities and Exchange Commission’s website at www.sec.gov. Information contained in or otherwise accessible through our website \n(www.bmo.com), or any third-party websites mentioned herein, does not form part of this document.'}]

Lets Create a text summarizer using Cosine Distance

In [ ]:
from nltk.corpus import stopwords
from nltk.cluster.util import cosine_distance
import numpy as np
import networkx as nx
import re

def read_article(article):
    sentences = []
    for sentence in article.split('. '):
        sentences.append(sentence.replace("[^a-zA-Z]", " ").split())
    return sentences

def sentence_similarity(sent1, sent2, stop_words=None):
    if stop_words is None:
        stop_words = []

    sent1 = [w.lower() for w in sent1 if w not in stop_words]
    sent2 = [w.lower() for w in sent2 if w not in stop_words]

    all_words = list(set(sent1 + sent2))

    vector1 = [0] * len(all_words)
    vector2 = [0] * len(all_words)

    # Build the vector for the first sentence
    for w in sent1:
        if w in stop_words:
            continue
        vector1[all_words.index(w)] += 1

    # Build the vector for the second sentence
    for w in sent2:
        if w in stop_words:
            continue
        vector2[all_words.index(w)] += 1

    return 1 - cosine_distance(vector1, vector2)

def build_similarity_matrix(sentences, stop_words):
    # Create an empty similarity matrix
    similarity_matrix = np.zeros((len(sentences), len(sentences)))

    for idx1 in range(len(sentences)):
        for idx2 in range(len(sentences)):
            if idx1 == idx2:  # ignore if both are the same sentence
                continue
            similarity_matrix[idx1][idx2] = sentence_similarity(sentences[idx1], sentences[idx2], stop_words)

    return similarity_matrix

def generate_summary(article, top_n=5):
    stop_words = stopwords.words('english')
    summarize_text = []

    # Step 1 - Read text and split it
    sentences = read_article(article)

    # Step 2 - Generate Similarity Matrix across sentences
    sentence_similarity_matrix = build_similarity_matrix(sentences, stop_words)

    # Step 3 - Rank sentences in similarity matrix
    sentence_similarity_graph = nx.from_numpy_array(sentence_similarity_matrix)
    scores = nx.pagerank(sentence_similarity_graph)

    # Step 4 - Sort the rank and pick top sentences
    ranked_sentence = sorted(((scores[i], s) for i, s in enumerate(sentences)), reverse=True)
    print("Indexes of top ranked_sentence order are ", ranked_sentence)

    for i in range(top_n):
        summarize_text.append(" ".join(ranked_sentence[i][1]))

    # Step 5 - Output the summarize text
    print("Summarized Text: \n", ". ".join(summarize_text))


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
generate_summary(all_extracted_texts[0] ['extracted_text'], 2)

Indexes of top ranked_sentence order are  [(0.5, ['Information', 'contained', 'in', 'or', 'otherwise', 'accessible', 'through', 'our', 'website', '(www.bmo.com),', 'or', 'any', 'third-party', 'websites', 'mentioned', 'herein,', 'does', 'not', 'form', 'part', 'of', 'this', 'document.']), (0.5, ['BMO’s', 'continuous', 'disclosure', 'materials,', 'including', 'interim', 'filings,', 'annual', 'Management’s', 'Discussion', 'and', 'Analysis', 'and', 'audited', 'annual', 'consolidated', 'financial', 'statements,', 'Annual', 'Information', 'Form', 'and', 'Notice', 'of', 'Annual', 'Meeting', 'of', 'Shareholders', 'and', 'Proxy', 'Circular,', 'are', 'available', 'on', 'our', 'website', 'at', 'www.bmo.com/investorrelations,', 'on', 'the', 'Canadian', 'Securities', 'Administrators’', 'website', 'at', 'www.sedarplus.ca,', 'and', 'on', 'the', 'EDGAR', 'section', 'of', 'the', 'U.S.', 'Securities', 'and', 'Exchange', 'Commission’s', 'website', 'at', 'www.sec.gov'])]
Summarized Text: 
 Information cont

Original Text : BMO’s continuous disclosure materials, including interim filings, annual Management’s Discussion and Analysis and audited annual consolidated \nfinancial statements, Annual Information Form and Notice of Annual Meeting of Shareholders and Proxy Circular, are available on our website at \nwww.bmo.com/investorrelations, on the Canadian Securities Administrators’ website at www.sedarplus.ca, and on the EDGAR section of the \nU.S.\xa0Securities and Exchange Commission’s website at www.sec.gov. Information contained in or otherwise accessible through our website \n(www.bmo.com), or any third-party websites mentioned herein, does not form part of this document.

Summarized Text : Information contained in or otherwise accessible through our website (www.bmo.com),
or any third-party websites mentioned herein, does not form part of this document..
BMO’s continuous disclosure materials, including interim filings, annual Management’s Discussion and Analysis and audited annual consolidated financial statements, Annual Information Form and Notice of Annual Meeting of Shareholders and Proxy Circular, are available on our website at www.bmo.com/investorrelations, on the Canadian Securities Administrators’ website at www.sedarplus.ca, and on the EDGAR section of the U.S. Securities and Exchange Commission’s website at www.sec.gov'

**What about the Tables?** Lets try PDFPlumber

In [ ]:
#!pip install pdfplumber

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.0/58.0 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 38.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 62.2 MB/s eta 0:00:00
  Attempting uninstall: pdfminer.six
    Found existing installation: pdfminer.six 20240706
    Uninstalling pdfminer.six-20240706:
      Successfully uninstalled pdfminer.six-20240706


In [ ]:
import pdfplumber
import pandas as pd

pdf_path = '/content/BMO Report.pdf'

# Open the PDF file
with pdfplumber.open(pdf_path) as pdf:

    page = pdf.pages[5] #BMO Report has table on Page 4

    text = page.extract_text()

print(text)

lines = text.split('\n')

data = []
headers = None

for line in lines:
    columns = line.split()

    if '(Canadian $ in millions, except as noted)' in line:
        headers = columns
    elif any(char.isdigit() for char in line.split()[0]):
        data.append(columns)

Summary of Reported and Adjusted Results by Operating Segment
BMO Wealth BMO Capital Corporate U.S. Segment (1)
(Canadian $ in millions, except as noted) Canadian P&C U.S. P&C Total P&C Management Markets Services Total Bank (US$ in millions)
Q2-2024
Reported net income (loss) 872 543 1,415 320 459 (328) 1,866 559
Acquisition and integration costs 2 - 2 - 2 22 26 17
Amortization of acquisition-related intangible assets 3 69 72 2 5 - 79 54
Legal provision (including related interest expense
and legal fees) - - - - - 12 12 9
Impact of FDIC special assessment - - - - - 50 50 37
Adjusted net income (loss) (2) 877 612 1,489 322 466 (244) 2,033 676
Q1-2024
Reported net income (loss) 921 560 1,481 240 393 (822) 1,292 184
Acquisition and integration costs 1 - 1 - 10 46 57 39
Amortization of acquisition-related intangible assets 3 75 78 1 5 - 84 59
Legal provision (including related interest expense
and legal fees) - - - - - 11 11 8
Impact of loan portfolio sale - - - - - 136 136 102
Impact of 

**Lets try PymuPDF**

In [ ]:
!pip install PyMuPDF

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.7/15.7 MB 31.1 MB/s eta 0:00:00


In [ ]:
import fitz
import pandas as pd

# Path to the PDF file
pdf_path = '/content/BMO Report.pdf'

document = fitz.open(pdf_path)
page = document[4]  # Page numbers are zero-indexed

# Extract text from the page
text = page.get_text("text")

# Print the extracted text
print(text)

Non-GAAP and Other Financial Measures 
(Canadian $ in millions, except as noted)
Q2-2024
Q1-2024
Q2-2023
YTD-2024
YTD-2023
Reported Results
Net interest income
4,515
4,721
4,814
9,236
8,835
Non-interest revenue
3,459
2,951
2,975
6,410
4,053
Revenue
7,974
7,672
7,789
15,646
12,888
Provision for credit losses
(705)
(627) 
(1,023) 
(1,332) 
(1,240) 
Non-interest expense
(4,844) 
(5,389) 
(5,501) 
(10,233) 
(9,883) 
Income before income taxes
2,425
1,656
1,265
4,081
1,765
Provision for income taxes
(559)
(364) 
(236) 
(923)
(603) 
Net income
1,866
1,292
1,029
3,158
1,162
Diluted EPS ($)
2.36
1.73
1.26
4.08
1.42
Adjusting Items Impacting Revenue (Pre-tax)
Management of fair value changes on the purchase of Bank of the West (1)
-
-
-
-
(2,011) 
Legal provision (recorded in revenue) (2)
(14)
(14) 
(7) 
(28)
(13) 
Impact of loan portfolio sale (3)
-
(164) 
-
(164)
-
Impact of adjusting items on revenue (pre-tax)
(14)
(178) 
(7) 
(192)
(2,024) 
Adjusting Items Impacting Provision for Credit Los

In [ ]:
!pip install tabula-py

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 67.1 MB/s eta 0:00:00


The show stopper : Tabula (Page 5)

In [ ]:
import locale

In [ ]:
# Function to set locale safely
def set_locale(locale_name):
    try:
        locale.setlocale(locale.LC_ALL, locale_name)
    except locale.Error:
        print(f"Locale {locale_name} not supported on this system. Using default locale.")
        locale.setlocale(locale.LC_ALL, '')

# Set the locale (adjust as necessary for your system)
set_locale('en_US.UTF-8')  # Common locale setting for Unix-like systems
locale._override_localeconv = {'n_sign_posn':1}

In [ ]:
#another way : Tabula

# Format as currency
def format_currency(value):
    if pd.isna(value):
        return ''
    return locale.currency(value, grouping=True).split('.')[0]

import tabula
# Extract tables from the PDF

# Path to the PDF file
pdf_path = '/content/BMO Report.pdf'
table = tabula.read_pdf(pdf_path, pages=5, multiple_tables=False)
# Check if a table was found
if table:
    for i, table_df in enumerate(table):

        existing_headers = table_df.columns
        table_df.columns = existing_headers

        table_df = table_df.drop(0)  # Drop the first row which is now the header
        table_df = table_df.reset_index(drop=True)  # Reset the index

         # Clean and process the DataFrame as needed
        numeric_columns = ['Q2-2024','Q1-2024','Q2-2023','YTD-2024','YTD-2023']

        for col in numeric_columns:
          table_df[col] = table_df[col].str.replace(',', '')
          table_df[col] = table_df[col].str.replace('(', '-')
          table_df[col] = table_df[col].str.replace(')', '')
          table_df[col] = pd.to_numeric(table_df[col], errors='coerce')
          table_df[col] = table_df[col].apply(format_currency)
else:
    print("No tables found.")

In [ ]:
table_df.head(30)

,"(Canadian $ in millions, except as noted)",Q2-2024,Q1-2024,Q2-2023,YTD-2024,YTD-2023
0,Net interest income,"$4,515","$4,721","$4,814","$9,236","$8,835"
1,Non-interest revenue,"$3,459","$2,951","$2,975","$6,410","$4,053"
2,Revenue,"$7,974","$7,672","$7,789","$15,646","$12,888"
3,Provision for credit losses,-$705,-$627,"-$1,023","-$1,332","-$1,240"
4,Non-interest expense,"-$4,844","-$5,389","-$5,501","-$10,233","-$9,883"
5,Income before income taxes,"$2,425","$1,656","$1,265","$4,081","$1,765"
6,Provision for income taxes,-$559,-$364,-$236,-$923,-$603
7,Net income,"$1,866","$1,292","$1,029","$3,158","$1,162"
8,Diluted EPS ($),$2,$1,$1,$4,$1
9,Adjusting Items Impacting Revenue (Pre-tax),,,,,


a solution we are looking for , But What about Page 6?

In [ ]:
#another way : Tabula - First Step

import tabula

# Path to the PDF file
pdf_path = '/content/BMO Report.pdf'
table = tabula.read_pdf(pdf_path, pages=6, multiple_tables=False)
# Check if a table was found
if table:
    for i, table_df in enumerate(table):

        existing_headers = table_df.columns
        new_headers = table_df.iloc[0]

        combined_headers = []

        for existing, new in zip(existing_headers, new_headers):
          combined_headers.append(f'{existing}_{new}')  # Combine existing and new headers

        table_df.columns = new_headers

        table_df = table_df.drop(0)  # Drop the first row which is now the header
        table_df = table_df.reset_index(drop=True)  # Reset the index

else:
    print("No tables found.")

In [ ]:
table_df.head(30)

,"(Canadian $ in millions, except as noted)",Canadian P&C,U.S. P&C,Total P&C,Management Markets,Services,Total Bank,NaN,(US$ in millions)
0,Q2-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Reported net income (loss),872,543,"1,415",320 459,(328),"1,866",NaN,559
2,Acquisition and integration costs,2,-,2,- 2,22,26,NaN,17
3,Amortization of acquisition-related intangible...,3,69,72,2 5,-,79,NaN,54
4,Legal provision (including related interest ex...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,and legal fees),-,-,-,- -,12,12,NaN,9
6,Impact of FDIC special assessment,-,-,-,- -,50,50,NaN,37
7,Adjusted net income (loss) (2),877,612,"1,489",322 466,(244),"2,033",NaN,676
8,Q1-2024,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Reported net income (loss),921,560,"1,481",240 393,(822),"1,292",NaN,184


Lets Improve Tabula Results

In [ ]:
import tabula
import pandas as pd
import numpy as np

# Path to the PDF file
pdf_path = '/content/BMO Report.pdf'

# Format as currency
def format_currency(value):
    if pd.isna(value):
        return ''
    return locale.currency(value, grouping=True).split('.')[0]

tables = tabula.read_pdf(pdf_path, pages=6, multiple_tables=False)

if tables:
    for i, table_df in enumerate(tables):

        # Assume the first row contains headers
        new_headers = table_df.iloc[0]
        table_df.columns = new_headers

        # Drop the first row which is now the header
        table_df = table_df.drop(0).reset_index(drop=True)

        if 'Management Markets' in table_df.columns:
            # Split the merged column into two separate columns
            split_cols = table_df['Management Markets'].str.split(expand=True)

            # Rename the new columns appropriately
            split_cols.columns = ['Management', 'Markets']

            # Drop the old merged column and concatenate the new columns
            table_df = table_df.drop(columns=['Management Markets'])
            table_df = pd.concat([table_df, split_cols], axis=1)

        # Clean and process the DataFrame as needed
        numeric_columns = ['Canadian P&C', 'U.S. P&C', 'Total P&C', 'Management', 'Markets', 'Services', 'Total Bank', '(US$ in millions)']

        for col in numeric_columns:
          table_df[col] = table_df[col].str.replace(',', '')
          table_df[col] = table_df[col].str.replace('(', '-')
          table_df[col] = table_df[col].str.replace(')', '')
          table_df[col] = pd.to_numeric(table_df[col], errors='coerce')
          table_df[col] = table_df[col].apply(format_currency)
else:
    print("No tables found.")

In [ ]:
actual_columns = ['(Canadian $ in millions, except as noted)','Canadian P&C', 'U.S. P&C', 'Total P&C', 'BMO Wealth Management', 'BMO Capital Markets', 'Corporate Services', 'Total Bank', 'U.S. Segment (US$ in millions)']

table_df.rename(columns={'Management': 'BMO Wealth Management', 'Markets': 'BMO Capital Markets', 'Services' : 'Corporate Services', '(US$ in millions)': 'U.S. Segment (US$ in millions)'}, inplace=True)
table_df = table_df[actual_columns]

In [ ]:
table_df.head(15)

,"(Canadian $ in millions, except as noted)",Canadian P&C,U.S. P&C,Total P&C,BMO Wealth Management,BMO Capital Markets,Corporate Services,Total Bank,U.S. Segment (US$ in millions)
0,Q2-2024,,,,,,,,
1,Reported net income (loss),$872,$543,"$1,415",$320,$459,-$328,"$1,866",$559
2,Acquisition and integration costs,$2,,$2,,$2,$22,$26,$17
3,Amortization of acquisition-related intangible...,$3,$69,$72,$2,$5,,$79,$54
4,Legal provision (including related interest ex...,,,,,,,,
5,and legal fees),,,,,,$12,$12,$9
6,Impact of FDIC special assessment,,,,,,$50,$50,$37
7,Adjusted net income (loss) (2),$877,$612,"$1,489",$322,$466,-$244,"$2,033",$676
8,Q1-2024,,,,,,,,
9,Reported net income (loss),$921,$560,"$1,481",$240,$393,-$822,"$1,292",$184


Some Final Touch up

In [ ]:
for col in table_df.columns:
    # Combine rows by indices for specific rows
    combined_row = table_df.iloc[4, table_df.columns.get_loc(col)] + ' ' + table_df.iloc[5, table_df.columns.get_loc(col)]
    combined_row = combined_row.replace('- -', '-')

    # Assign the combined row to the DataFrame
    table_df.iloc[4, table_df.columns.get_loc(col)] = combined_row

# Drop the redundant row
table_df = table_df.drop(index=5).reset_index(drop=True)

for col in table_df.columns:
    # Combine rows by indices for specific rows
    combined_row = table_df.iloc[11, table_df.columns.get_loc(col)] + ' ' + table_df.iloc[12, table_df.columns.get_loc(col)]
    combined_row = combined_row.replace('- -', '-')

    # Assign the combined row to the DataFrame
    table_df.iloc[11, table_df.columns.get_loc(col)] = combined_row

# Drop the redundant row
table_df = table_df.drop(index=12).reset_index(drop=True)

In [ ]:
table_df.head(15)

,"(Canadian $ in millions, except as noted)",Canadian P&C,U.S. P&C,Total P&C,BMO Wealth Management,BMO Capital Markets,Corporate Services,Total Bank,U.S. Segment (US$ in millions)
0,Q2-2024,,,,,,,,
1,Reported net income (loss),$872,$543,"$1,415",$320,$459,-$328,"$1,866",$559
2,Acquisition and integration costs,$2,,$2,,$2,$22,$26,$17
3,Amortization of acquisition-related intangible...,$3,$69,$72,$2,$5,,$79,$54
4,Legal provision (including related interest ex...,,,,,,$12,$12,$9
5,Impact of FDIC special assessment,,,,,,$50,$50,$37
6,Adjusted net income (loss) (2),$877,$612,"$1,489",$322,$466,-$244,"$2,033",$676
7,Q1-2024,,,,,,,,
8,Reported net income (loss),$921,$560,"$1,481",$240,$393,-$822,"$1,292",$184
9,Acquisition and integration costs,$1,,$1,,$10,$46,$57,$39
